# Effective Python Testing with Pytest

## What Makes `pytest` Useful?
With `pytest`, common tasks require less code and advanced tasks can be achieved through a variety of time-saving commands and plugins. It also runs existing tests out of the box.

This tutorial helps to understand some tools `pytest` provides to keep testing efficient and effective even as it scales.

### Less Boilerplate
Most functional tests follow the Arrange-Act-Assert model:
- **Arrange**: setup the conditions for the test
- **Act**: call some function or method
- **Assert**: assert that some end condition is true

Testing frameworks typically hook into your test's assertions so that they can provide information when an assertioin fails. Even a small set of tests requires a fair amount of boilerplate code.

```python
from unittest import TestCase

class TryTesting(TestCase):
    def test_always_passes(self):
        self.assertTrue(True)
    
    def test_always_fails(self):
        self.assertTrue(False)
```

These tests can be run from the command line using the `discover` option of `unittest`:

`python -m unittest discover`

A lot had to be done to run this test:
- import `TestCase` from `unittest`
- create `TryTesting` subclass
- write a method in `TryTesting` for each test
- use one of the `self.assert*` methods from `unittest.TestCase` to make assertions

This is a significant amount of code to write, and because it's the minimum you need for *any* test, you'd end up writing the same code over and over. `pytest` simplifies this workflow by allowing you to use Python's `assert` keyword directly:

In [7]:
# test_with_pytest.py

def test_always_passes():
    assert True
    
def test_always_fails():
    assert False

If you can write an expression that you expect to evaluate to `True`, then pytest will test it for you. It can be run using the `pytest` command:

In [9]:
!pytest

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 2 items                                                              

test_with_pytest.py .F                                                   [100%]

=================================== FAILURES ===================================
______________________________ test_always_fails _______________________________

    def test_always_fails():
>       assert False
E       assert False

test_with_pytest.py:5: AssertionError
=========================== short test summary info ============================
FAILED test_with_pytest.py::test_always_fails - assert False
========================= 1 failed, 1 passed in 0.13s ==========================


The `pytest` report shows:
- the system state, including which versions of Python, pytest, and any plugins you have installed
- the rootdir, the directory to search under for configuration and tests
- the number of test the runner discovered

The output then indicates the status of each test:
- **dot (.)** means the test passed
- **F** means the test failed
- **E** means the test raised an unexpected exception

For tests that fail, the report gives a detailed breakdown of the failure. In the example above, the test failed because `assert False` always fails. Finally, the report gives an overall status report of the test suite.

A few more assertion examples:

In [10]:
# test_with_pytest.py

def test_uppercase():
    assert "loud noises".upper() == "LOUD NOISES"
    
def test_reversed():
    assert list(reversed([1, 2, 3, 4])) == [4, 3, 2, 1]
    
def test_some_primes():
    assert 37 in {
        num
        
        for num in range(1, 50)
        if num != 1 and not any(
            [num % div == 0 for div in range(2, num)]
        )
    }

In [11]:
!pytest

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 5 items                                                              

test_with_pytest.py .F...                                                [100%]

=================================== FAILURES ===================================
______________________________ test_always_fails _______________________________

    def test_always_fails():
>       assert False
E       assert False

test_with_pytest.py:6: AssertionError
=========================== short test summary info ============================
FAILED test_with_pytest.py::test_always_fails - assert False
========================= 1 failed, 4 passed in 0.15s ==========================


The learning curve for `pytest` is shallower than for `unittest` because you don't need to learn new constructs for most tests. Also, the use of `assert` makes tests more understandable.

### State and Dependency Management
Tests should help you make your code more understandable. If the tests themselves are difficult to understand, then you may be in trouble.

`pytest` leads you toward **explicit** dependency declarations that are still reusable thanks to the availability of [fixtures](https://docs.pytest.org/en/latest/fixture.html). `pytest` fixtures are functions that create data or test doubles or initialize some system state for the test suite. Any test that wants to use a fixture must explicitly accept it as an argument, so dependencies are always stated up front.

Fixtures can also make use of other fixtures, again by declaring them explicitly as dependencies. Over time, your fixtures can become bulky and modular. Although the ability to insert fixtures into other fixtures provides enormous flexibility, it can also make managing dependencies more challenging as your test suite grows.

### Test Filtering
As your test suite grows, you may want to run just a few tests on a feature and save the full suite for later. `pytest` provides a few ways of doings this:
- **Name-based filtering**: You can limit pytest to running only those tests whose fully qualified names match a particular expression. You can do this with the `-k` parameter.
- **Directory scoping**: By default, `pytest` will run only those tests that are in or under the current directory
- **Test categorization**: `pytest` can include or exclude tests from particular categories that you define. You can do this with the `-m` parameter.

`pytest` enables you to create **marks**, or custom labels, for any test you like. A test may have multiple labels, and you can use them for granular control over which tests to run.

### Test Parameterization
When testing functions that process data or perform generic transofrmatoins, you'll be writing many similar tests. They may differ only in input or output of the code being tested. This requires duplicate test code, and doing so can sometimes obscure the behavior you're trying to test.

`pytest` offers a solution in which each test can pass or fail independently.

### Plugin-Based Architecture
`pytest` has a rich ecosystem of helpful plugins.

## Fixtures: Managing State and Dependencies
`pytest` fixtures are a way of providing data, test doubles, or state setup to your tests. Fixtures are functions that can return a wide range of values. Each test that depends on a fixture must explicitly accept that fixture as an argument.

### When to Create Fixtures
Imagine you're writing a function, `format_data_for_display()`, to process the data returned by an API endpoint. The data represents a list of people, each with a given name, family name, and job title. The function should output a list of strings that include each person's full name (their `given_name` followed by their `family_name`), a colon, and their `title`. To test this, you might write the following code:

In [1]:
# format_data.py

def format_data_for_display(people):
    '''
    Formats data returned from an API endpoint.
    
    Parameters
    ----------
    people : list
        A list of people with a given name, family name, and
        job title
        
    Returns
    -------
    list
        A list of strings formatted to include the person's full
        name (given name followed by family name) and their title.
    '''
    return [
        f"{person['given_name']} {person['family_name']}:"
        f" {person['title']}"
        
        for person in people
    ]


def test_format_data_for_display():
    '''
    Tests format_data_for_display function.
    '''
    people = [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

    assert format_data_for_display(people) == [
        "Alfonsa Ruiz: Senior Software Engineer",
        "Sayid Khan: Project Manager",
    ]

In [3]:
!pytest -s format_data.py

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 1 item                                                               

format_data.py .

============================== 1 passed in 0.01s ===============================


Now suppose you need to write another function that transofrms the data into comma-separated values for use in Excel. The test looks familiar to the above:

In [14]:
# format_data_csv.py

def format_data_for_excel(people):
    '''
    Formats data returned from an API endpoint.
    
    Parameters
    ----------
    people : list
        A list of people with a given name, family name, and
        job title
        
    Returns
    -------
    str
        A string formatted to include the person's full
        name (given name followed by family name) and their title
        in comma-separated values.
    '''
    header = ','.join(people[0].keys())
    values = [','.join(person.values()) for person in people]
    
    # rejoin with new lines
    values = '\n'.join(values)
    
    return f"{header}\n{values}"

def test_format_data_for_excel():
    people = [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

    assert format_data_for_excel(people) == (
        "given_name,family_name,title\n"
        "Alfonsa,Ruiz,Senior Software Engineer\n"
        "Sayid,Khan,Project Manager"
    )

In [15]:
!pytest -s format_data_csv.py

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 1 item                                                               

format_data_csv.py .

============================== 1 passed in 0.01s ===============================


If you find yourself writing several tests that all make use of underlying test data, then a fixture may be what you want. You can pull repeated data into a single function decorated with `@pytest.fixture` to indicate that the function is a `pytest` fixture:

In [16]:
import pytest

In [17]:
# data_format_fixture.py

@pytest.fixture
def example_people_data():
    return [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

We can use this fixture by adding it as an argument to tests. Its value will be the return value of the fixture function:

In [19]:
# data_format_fixture.py

def test_format_data_for_display(example_people_data):
    assert format_data_for_display(example_people_data) == [
        "Alfonsa Ruiz: Senior Software Engineer",
        "Sayid Khan: Project Manager",
    ]

def test_format_data_for_excel(example_people_data):
    assert format_data_for_excel(example_people_data) == (
        "given_name,family_name,title\n"
        "Alfonsa,Ruiz,Senior Software Engineer\n"
        "Sayid,Khan,Project Manager"
    )

In [21]:
!pytest -s data_format_fixture.py

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 2 items                                                              

data_format_fixture.py ..

============================== 2 passed in 0.01s ===============================


Each test is now notably shorter but still has a clear path back to the data it depends on. Be sure to name fixtures something specific. That way it is easily determined if you want to use it when writing new tests in the future.

### When to Avoid Fixtures
Fixtures aren't great for extracting data or objects that you use across multiple tests. They aren't always as good for tests that require slight variations in the data. Littering your test suite with fixtures is no better than littering it with plain data or objects. It might even be worse because of the added layer of indirection.

As with most abstraction, it takes some practice and thought to find the right level of fixture use.

### Fixtures at Scale
As you extract more fixtures from your tests, you might see that some fixtures could benefit from further extraction. Fixtures are **modular**, so they can depend on other fixtures. You may find that fixtures in two separate test modules share a common dependency.

You can move fixtures from test modules into more general fixture-related modules. That way, you can import them back into any test modules that need them. This is a good approach when you find yourself using a fixture repeatedly throughout your project.

pytest looks for `conftest.py` modules throughout the directory structure. Each `conftest.py` provides configuration for the file tree pytest finds in it. You can use any fixtures that are defined in a particular `conftest.py` throughout the file's parent directory and in any subdirectories. This is a great place to put your most widely used fixtures.

Another interesting use case for fixtures is in guarding access to resources. Imagine writing a test suite for code that deals with API calls. You want to ensure that the test suite doesn't make any real network calls, even if a test accidentally executes the real network call code. `pytest` provides a [monkeypatch](https://docs.pytest.org/en/latest/monkeypatch.html) fixture to replace values and behaviors, which you can use to great effect:

In [23]:
# conftest.py

import pytest
import requests


@pytest.fixture(autouse=True)
def disable_network_calls(monkeypatch):
    def stunted_get():
        raise RuntimeError(
            "Network access not allowed during testing!"
        )
        
    monkeypatch.setattr(
        requests,
        "get",
        lambda *args, **kwargs: stunted_get()
    )

By placing `disable_network_calls()` in `conftest.py` and adding the `autouse=True` option, you ensure that network calls will be disabled in every test across the suite. Any test that executes code calling `requests.get()` will raise a `RuntimeError` indicating that an unexpected network call would have occurred.

## Marks: Categorizing Tests
In any large test suite, some tests will inevitably be slow. They might test timeout behavior, or they might exercise a broad area of the code. Whatever the reason, it would be nice to avoid running *all* the slow tests when trying to iterate quickly on a new feature.

`pytest` enables you to define categories for your tests and provides options for including or excluding categories when you run your suite. You can mark a test with any number of categories.

Marking tests is useful for categorizing tests by subsystem or dependencies. If some tests require access to a database, for example, you could create a `@pytest.mark.database_access` mark for them.

> **PRO TIP**: Because you can give your marks any name you want, it can be easy to mistype or misremember the name of a mark. pytest will warn you about marks that it doesn't recognize.
>
> The `--strict-markers` flag to the pytest command ensures that all marks in your tests are registered in your pytest configuration. It will prevent you from running your tests until you register any unknown marks.
>
> Checkout [registering marks](https://docs.pytest.org/en/latest/mark.html#registering-marks) for more info.

When the time comes to run tests, you can still run them all by default with the `pytest` command. If you'd like to run only those tests that require database access, then you can use `pytest -m database_access`. To run all tests *except* those that require database access, you can use `pytest -m "not database_access"`. You can even use an `autouse` fixture to limit database access to those tests marked with `database_access`.

Some plugins expand on the functionality of marks by guarding access to resources.

`pytest` provides a few marks out of the box:
- **`skip`**: skips a test unconditionally
- **`skipif`**: skips a test if the expression passed to it evaluates to `True`
- **`xfail`**: indicates that a test is expected to fail, so if the test *does* fail, the overall suite can still result in a passing status
- **`parametrize`**: (note spelling) creates multiple variants of a test with different values as arguments

You can see a list of all marks pytest knows by running `pytest --markers`.

## Parametrization: Combining Tests
Fixtures aren't quite as useful when you have several tests with slightly different inputs and expected outputs. In these cases, you can **parametrize** a single test definitoin, and `pytest` will create variants of the test for you with the parameters you specify.

Imagine you've written a function to tell if a string is a palindrome. An initial set of tests could look like:

In [24]:
def test_is_palindrome_empty_string():
    assert is_palindrome("")

    
def test_is_palindrome_single_character():
    assert is_palindrome("a")

    
def test_is_palindrome_mixed_casing():
    assert is_palindrome("Bob")

    
def test_is_palindrome_with_spaces():
    assert is_palindrome("Never odd or even")

    
def test_is_palindrome_with_punctuation():
    assert is_palindrome("Do geese see God?")

    
def test_is_palindrome_not_palindrome():
    assert not is_palindrome("abc")

    
def test_is_palindrome_not_quite():
    assert not is_palindrome("abab")

All of these tests excep the last two have the same shape:

```python
def test_is_palindrome_<in some situation>():
    assert is_palindrome("<some string>")
```

You can use `@pytest.mark.parametrize()` to fill in this shape with different values, reducing your test code significantly:

In [25]:
@pytest.mark.parametrize("palindrome", [
    "",
    "a",
    "Bob",
    "Never odd or even",
    "Do geese see God?",
])
def test_is_palindrome(palindrome):
    assert is_palindrome(palindrome)
    

@pytest.mark.parametrize("non_palindrome", [
    "abc",
    "abab",
])
def test_is_palindrome_not_palindrome(non_palindrome):
    assert not is_palindrome(non_palindrome)

The first argument to `parametrize()` is a comma-delimited string of parameter names. The second argument is a list of either tuples or single values that represent the parameter value(s).

You can take your parametrization a step further to combine all your tests into one:

In [26]:
@pytest.mark.parametrize(
    "maybe_palindrome, expected_result",
    [
        ("", True),
        ("a", True),
        ("Bob", True),
        ("Never odd or even", True),
        ("Do geese see God?", True),
        ("abc", False),
        ("abab", False),
    ]
)
def test_is_palindrome(maybe_palindrome, expected_result):
    assert is_palindrome(maybe_palindrome) == expected_result

> **NOTE**: Even though this shortened the code, in this case, it didn't do much to clarify your test code. Use parametrization to separate the test data from the test behavior so that it's clear what the test is testing!

## Durations Reports: Fighting Slow Tests
Each time you switch contexts from implementation code to test code, you incur some overhead. If your tests are slow to begin with, then overhead can cause friction and frustration.

If you want to improve the speed of your tests, then it's useful to know which tests might offer the biggest improvements. `pytest` can automatically record test durations for you and report the top offenders.

Use the `--durations` option to the `pytest` command to include a duration report in your test results. `--durations` expects an integer value `n` and will report the slowest n number of tests.

The output will follow test results:

```shell
$ pytest --durations=3
3.03s call     test_code.py::test_request_read_timeout
1.07s call     test_code.py::test_request_connection_timeout
0.57s call     test_code.py::test_database_read
======================== 7 passed in 10.06s ==============================
```

Each test that shows up in the durations report is a good candidate to speed up because it takes an above-average amount of the total testing time.

Be aware that some tests may have an invisible setup overhead.

## Useful `pytest` Plugins


### `pytest-randomly`
[`pytest-randomly`](https://github.com/pytest-dev/pytest-randomly) forces your tests to run in a random order. `pytest` always collects all the tests it can find before running them, so `pytest-randomly` shuffles that list of tests just before execution.

This is a great way to uncover tests that depend on running in a specific order, which means they have a **stateful dependency** on some other test. If you built your test suite from scratch in pytest, then this isn't very likely. It's more likely to happen in test suites that you migrate to pytest.

The plugin will print a seed value in the configuration description. You can use that value to run the tests in the same order as you try to fix the issue.

### `pytest-cov`
If you measure how well your tests cover your implementation code, you likely use the [coverage](https://coverage.readthedocs.io/) package. [`pytest-cov`](https://pytest-cov.readthedocs.io/en/latest/) integrates coverage, so you can run `pytest --cov` to see the test coverage report.

### `pytest-django`
[`pytest-django`](https://pytest-django.readthedocs.io/en/latest/) provides a handful of useful fixtures and makrs for dealing with Django tests. The `rf` fixture provides direct access to an instance of Django's [RequestFactory](https://docs.djangoproject.com/en/3.0/topics/testing/advanced/#django.test.RequestFactory). The `setting`